In [5]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [6]:
df = pd.read_csv('/content/Car_insurance_data.csv')
df.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_UR3VUI0,2/12/2010,2/11/2011,Male,62,2/12/2010,1,Saloon,Silver,TOYOTA,Anthony Village,Lagos,Car Classic,0
1,ID_BVMENF3,7/10/2010,1/23/2011,Female,6,7/10/2010,1,Saloon,Silver,Mercedes,Lagos Island,Lagos,Car Classic,0
2,ID_BHFL6A6,7/10/2010,1/23/2011,Male,6,7/10/2010,1,Saloon,Green,Mercedes,Lagos Island,Lagos,Car Classic,0
3,ID_8M0PZF5,1/22/2010,1/21/2011,Female,44,1/22/2010,2,Saloon,Black,TOYOTA,Lekki,Lagos,Car Classic,0
4,ID_IH42QHY,3/28/2010,3/27/2011,Male,57,3/28/2010,1,Pick Up,White,Honda,SANGO OTTA,Ogun,Car Classic,0


In [8]:
df.drop('Subject_Car_Colour', axis =1)

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_UR3VUI0,2/12/2010,2/11/2011,Male,62,2/12/2010,1,Saloon,TOYOTA,Anthony Village,Lagos,Car Classic,0
1,ID_BVMENF3,7/10/2010,1/23/2011,Female,6,7/10/2010,1,Saloon,Mercedes,Lagos Island,Lagos,Car Classic,0
2,ID_BHFL6A6,7/10/2010,1/23/2011,Male,6,7/10/2010,1,Saloon,Mercedes,Lagos Island,Lagos,Car Classic,0
3,ID_8M0PZF5,1/22/2010,1/21/2011,Female,44,1/22/2010,2,Saloon,TOYOTA,Lekki,Lagos,Car Classic,0
4,ID_IH42QHY,3/28/2010,3/27/2011,Male,57,3/28/2010,1,Pick Up,Honda,SANGO OTTA,Ogun,Car Classic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,ID_TWZXZPN,6/1/2010,5/31/2011,Male,45,6/1/2010,1,Bus,Volkswagen,Victoria Island,Lagos,CVTP,0
2743,ID_UNN5XAF,7/23/2010,4/7/2011,Female,45,7/23/2010,1,Bus,Ford,Victoria Island,Lagos,CVTP,0
2744,ID_O8BGMOY,6/6/2010,6/5/2011,Male,45,6/6/2010,1,Bus,TOYOTA,Victoria Island,Lagos,CVTP,0
2745,ID_1B8HT3N,1/24/2010,1/23/2011,Female,39,1/24/2010,1,Bus,Ford,Victoria Island,Lagos,CVTP,0


In [9]:
#1. What is the distribution of the target variable

frequency = df['target'].value_counts()
percentage = df['target'].value_counts()*100/len(df)
result = pd.concat([frequency, percentage], axis=1)
print("Frequency and Percentage Distribution of Target:")
result.columns = ['frequency', 'percentage']
print(result)

Frequency and Percentage Distribution of Target:
        frequency  percentage
target                       
0            2240   81.543502
1             507   18.456498


In [10]:
#2. Correlation of age to target

df['Age'].corr(df['target'])

-0.13212810698756136

In [11]:
#3. Average No_Pol (number of policies) per customer

df['No_Pol'].mean()

1.4899890789952677

In [12]:
#4. Differences in policy outcomes (target) across different ProductName

Differences = df.groupby('ProductName')['target'].value_counts().unstack(fill_value=0).sort_values(by=1, ascending=False)
Differences.columns = ['No Claim', 'Claim']
print(Differences)

                  No Claim  Claim
ProductName                      
Car Classic           1827    420
Car Plus               119     39
CVTP                    88     32
Customized Motor       175     13
CarFlex                  4      3
Car Vintage              1      0
CarSafe                 21      0
Motor Cycle              4      0
Muuve                    1      0


In [13]:
#5.	What is the average duration between Policy Start Date and Policy End Date?

df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'])
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'])
df['Policy Duration'] = df['Policy End Date'] - df['Policy Start Date']
average_duration = df['Policy Duration'].mean()
average_duration

Timedelta('352 days 14:38:34.379322896')

In [14]:
#6.	Does the Car_Category affect the outcome of the target variable?

Car_Category = pd.crosstab(df['Car_Category'], df['target']).sort_values(by=1, ascending=False)
Car_Category
chi2, p, _, _ = chi2_contingency(Car_Category)
print(Car_Category.head())
print(f"Chi-square test statistic: {chi2}, p-value: {p}")

if p < 0.05:
    print("Conclusion: Yes, car category affects the target outcome.")
else:
    print("Conclusion: There is no significant relationship between them.")

target           0    1
Car_Category           
Saloon        1223  324
JEEP           888  143
Truck           56   22
Mini Bus        12    6
Bus             26    5
Chi-square test statistic: 34.691994400686205, p-value: 0.0009439436809089054
Conclusion: Yes, car category affects the target outcome.


In [16]:
#7. How does Car_Make (e.g., TOYOTA, etc.) impact policy outcomes?

Car_dist = pd.crosstab(df['Subject_Car_Make'], df['target']).sort_values(by=1, ascending=False)
Car_dist
chi2, p, _, _ = chi2_contingency(Car_dist)
print(Car_dist.head())
print(f"Chi-square test statistic: {chi2}, p-value: {p}")

if p < 0.05:
    print("Conclusion: People who use Toyota car are likely to claim compared to others")
else:
    print("Conclusion: No significant relationship was found between Subject_Car_Make and target outcome.")

target               0    1
Subject_Car_Make           
TOYOTA            1106  301
Honda              239   45
Lexus              177   32
Mercedes           133   26
Hyundai            133   19
Chi-square test statistic: 57.84057151399363, p-value: 0.0787511740044402
Conclusion: No significant relationship was found between Subject_Car_Make and target outcome.


In [17]:
#8. How is gender distributed?

print(df['Gender'].value_counts())

Gender
Male            1726
Female           875
Entity            90
Joint Gender      56
Name: count, dtype: int64


In [18]:
#9.	How does the No_Pol correlate with the customer’s age?

df['No_Pol'].corr(df['Age'])

0.05388397657091712

In [19]:
#10. What is the average age of users

df['Age'].mean()

43.418274481252276

In [20]:
#11.	What is the gender distribution in the dataset, and how does it relate to policy outcomes?

Gender = pd.crosstab(df['Gender'], df['target']).sort_values(by=1, ascending=False)
Gender
chi2, p, _, _ = chi2_contingency(Gender)
print(Gender)
print(f"Chi-square test statistic: {chi2}, p-value: {p}")

if p < 0.05:
    print(" Conclusion: There is a significant relationship between them as male are more likely to claim.")
else:
    print("Conclusion: No statistical significant relationship was found between Gender and target outcome.")

target           0    1
Gender                 
Male          1392  334
Female         741  134
Entity          62   28
Joint Gender    45   11
Chi-square test statistic: 16.28696298475456, p-value: 0.0009902544371628785
 Conclusion: There is a significant relationship between them as male are more likely to claim.


In [23]:
#12. Unique car categories and their distribution

unique_categories, distribution = df['Car_Category'].nunique(), df['Car_Category'].value_counts()
print(f"Number of unique car categories: {unique_categories}")
print("Distribution of car categories:")
print(distribution.head())

Number of unique car categories: 14
Distribution of car categories:
Car_Category
Saloon     1547
JEEP       1031
Truck        78
Bus          31
Pick Up      19
Name: count, dtype: int64


In [24]:
#13.	Does the length of time between the First Transaction Date and Policy End Date affect the outcome

df['Policy End Date'] = pd.to_datetime(df['Policy End Date'])
df['First Transaction Date'] = pd.to_datetime(df['First Transaction Date'])
df['Transaction_to_End_Duration'] = df['Policy End Date'] - df['First Transaction Date']
df['Transaction_to_End_Duration'] = df['Transaction_to_End_Duration'].dt.days # Convert to days
correlation = df['Transaction_to_End_Duration'].corr(df['target'])
print(f"Correlation between Transaction to End Duration and Target: {correlation}")

Correlation between Transaction to End Duration and Target: -0.017513783581276927


In [25]:
#14. Percentage of policies for Saloon cars compared to other categories

(df['Car_Category'] == 'Saloon').mean() * 100

56.31598107025846

In [26]:
#15.	Are customers with multiple policies (No_Pol > 1) more likely to have certain outcomes

multiple_policies_df = df[df['No_Pol'] > 1]
frequency_table = multiple_policies_df['target'].value_counts().reset_index()
print("Target and Frequency for Multiple Policies:")
print(frequency_table)

Target and Frequency for Multiple Policies:
   target  count
0       0    777
1       1    165
